In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib

from src.model import tscv
from src.feature_engineering import get_feature_cols

%run constants.py

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.1
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2


# Feature importances

Our models are not improving that greatly from adding the features as I thought they would and our best result was using just the mean encodings. Because of that I decided to further investigate it by looking into feature importances.

Firstly let's load the model we'll study and the feature set we used to train it.

In [ ]:
train_set = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set-features-005.parquet'))
reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-features-005.model'))

In [ ]:
reg

In [ ]:
train_set.describe()

In [ ]:
reg.feature_importances_

In [ ]:
sort_idx = np.flip(np.argsort(reg.feature_importances_))

sns.barplot(x=reg.feature_importances_[sort_idx], y=get_feature_cols(train_set)[sort_idx])